<h1>Estimating implied time scales with bootstrapping<h1>

In [ ]:
from msmbuilder.msm import MarkovStateModel
from multiprocessing import Pool
from itertools import repeat
import pandas as pd
import numpy as np
import os

os.makedirs('./microstate_MSM_and_PCCA',exist_ok=True)

def at_lagtime(lt,choice):
    """
    Calculate the implied timescales from a series of MSM lag time, based on the choice of MD trajectories

    Parameters
    ----------
    lt: list
        A list consists of MSM lag time

    choice: list
        A list consists of choice of MD trajectories

    Return
    ----------
    df: pandas dataframe
        a pandas dataframe containing the implied timescales at different lag time

    """
    clustering=np.load("./clustering/no_features/24/clustering_assignments.npy")
    clustering_sequence=[]
    for i in choice:
        clustering_sequence.append(clustering[i])
    df=pd.DataFrame()
    for o,n in enumerate(lt):
        msm = MarkovStateModel(lag_time=n, n_timescales=4, reversible_type="transpose", verbose=False,ergodic_cutoff="off")
        msm.fit(clustering_sequence)
        timescales={}
        timescales['lag_time']=[]
        timescales['lag_time'].append(n)
        for n in range(msm.n_timescales):
            timescales['timescale_{}'.format(n)] =[]
        for n in range(msm.n_timescales):
            timescales['timescale_{}'.format(n)].append(msm.timescales_[n])
        lt_df  = pd.DataFrame(timescales, columns=timescales.keys())
        df = pd.concat([df, lt_df],ignore_index=True)
    return df

def its_bootstrap(lt,bootstrap_no,msm_dir='./microstate_MSM_and_PCCA/'):
    """
   A Wrapper for doing boostrapping on the implied time scales

    Parameters
    ----------
    lt: list
        A list consists of MSM lag time

    bootstrap_no: int
        Number of subsampling to be done

    msm_dir: str
        Directory for saving the output

    Return
    ----------
    df: pandas dataframe
        a pandas dataframe containing the implied timescales at different lag time after bootstraping

    """
    bootstrap=[]
    traj=np.arange(bootstrap_no)
    os.makedirs("{}choice/".format(msm_dir),exist_ok=True)
    pool = Pool()
    if __name__ == '__main__':
#         while True:
#             try:
        for i in range(bootstrap_no):
            choice=np.random.choice(traj, bootstrap_no, replace=True).tolist()
            bootstrap.append(choice)
            np.savetxt("{}choice/choice{}.txt".format(msm_dir,i),choice,fmt='%i')
        dfs=pool.starmap(at_lagtime, zip(repeat(lt),bootstrap))
#             except ValueError:
#                 print('Rerunning due to NaN in transition counts')
#                 pass
#             else:
#                 break
    data = pd.concat(dfs, ignore_index=True)
    return data

msm_dir='./microstate_MSM_and_PCCA/'

lagtimes=[1]
for i in range(10,210,10):
    lagtimes.append(i)

no_of_traj=100

#You may want to rerun this cell if you encountered a ValueError
#Running this cell might take more than an hour on a typical Workstation
data=its_bootstrap(lagtimes,no_of_traj)
data.to_csv(msm_dir+'timescales.txt',index=False,sep="\t",float_format='%g')
data.to_pickle(msm_dir+'timescales.pickl')

<h1>Plotting the implied time scales against lag time<h1>

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
msm_dir='./microstate_MSM_and_PCCA/'
import pickle

def load_its(data,n_timescales,lagtimes):
    """
    Caculating the mean and the standard deviation for implied time scales 
    Parameters
    ----------
    data: pandas dataframe
        A list consists of MSM lag time
        
    n_timescales: int
        Number of implied timescales caluclated 
    
    lagtimes: list
        A list containing MSM lag time for plotting the implied timescales
        
    Return
    ----------
    av_dict: dict
        a dictionary containing the mean implied timescales
        
    error_dict: dict
        a dictionary containing the standard deviation of implied timescales
    
    """
#     data=pd.read_pickle(data)
    with open(str(data), 'rb') as f:
        data = pickle.load(f)
    dict_={}
    av_dict={}
    error_dict={}
    for i in range(n_timescales):
        dict_[i]=[]
        av_dict[i]=[]
        error_dict[i]=[]

    for m,i in enumerate(lagtimes):
        temp=data.query("lag_time == '{}'".format(i))
        for n in range(4):
            dict_[n].append(temp["timescale_{}".format(n)].values.tolist())
            av_dict[n].append(np.average(dict_[n][m]))
            error_dict[n].append(np.std(dict_[n][m]))
    return av_dict, error_dict


def plot_timescales(ax,n_timescales,lagtime,av_dict,error_dict,dt):
    """
    Plotting the mean and standard deviation for implied time scales 
    Parameters
    ----------
    ax: matplotlib.axis Class object
        The matplotlib.axis to plot 
    
    n_timescales: int
        Number of implied timescales caluclated 
    
    lagtimes: list
        A list containing MSM lag time for plotting the implied timescales
        
    av_dict: dict
        a dictionary containing the mean implied timescales
        
    error_dict: dict
        a dictionary containing the standard deviation of implied timescales
    
    """
    colors=['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple',
        'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']
    for i,color in zip(range(n_timescales),colors):
        ax.plot(np.array(lagtimes)*dt,
                   np.array(av_dict[i])*dt,
                   markersize=3,linewidth=2,linestyle='solid', c=color,
                   )
        ax.fill_between(np.array(lagtimes)*dt, 
                        (np.array(av_dict[i])-np.array(error_dict[i]))*dt,
                        (np.array(av_dict[i])+np.array(error_dict[i]))*dt,
                        color=color,alpha=0.2)
    xmin, xmax = ax.get_xlim()
    xx = np.linspace(xmin, xmax)
    ax.set_xlabel('Lag Time (ps) ', fontsize=20)
    ax.set_ylabel('Implied Timescales (ps) ', fontsize=20)
    #ax.set_yscale('log')
    plt.xticks(size=15)
    plt.yticks(size=15)
    #plt.ylim(bottom=0,top=100)
    plt.tight_layout()
    plt.savefig(msm_dir+'its_plot.png',transparent=True)

msm_dir='./microstate_MSM_and_PCCA/'

lagtimes=[1]
for i in range(10,210,10):
    lagtimes.append(i)
    
#Plotting implied timescales against lag time
fig, ax = plt.subplots(figsize=(7,5))
av_dict, error_dict=load_its(msm_dir+'timescales.pickl',n_timescales=4,lagtimes=lagtimes)
plot_timescales(ax,4,lagtimes,av_dict,error_dict,dt=0.1)




<h1>Lumping with PCCA+<h1>

In [ ]:
#Performing Lumping with PCCA+
from msmbuilder.lumping import PCCAPlus
from msmbuilder.msm import MarkovStateModel
import numpy as np
from msmbuilder.io import save_generic
import os

#Load clustering assignment
msm_dir='./microstate_MSM_and_PCCA/'
qmsm_dir='./qMSM/'
assignments=np.load("./clustering/no_features/24/clustering_assignments.npy")

#Generating the microstate TPM at lag time=10ps for lumping
msm=MarkovStateModel(verbose=True, lag_time=100, reversible_type='transpose', ergodic_cutoff='on')
msm_assignments=msm.fit_transform(assignments)
save_generic(msm,"{}msm.pickl".format(msm_dir))

np.savetxt(msm_dir+"microstate_TCM.txt",msm.countsmat_)
np.savetxt(msm_dir+"microstate_TPM.txt", msm.transmat_)
np.savetxt(msm_dir+"microstate_Population.txt", msm.populations_)

#lumping
pcca = PCCAPlus.from_msm(msm, n_macrostates=4)
lumped_trajs = pcca.fit_transform(assignments)
lumped_trajs=np.concatenate(lumped_trajs)
save_generic(pcca,"{}pcca.pickl".format(msm_dir))
np.save("{}lumping_assignment.npy".format(qmsm_dir),lumped_trajs)


<h1>Visualizing Metastable states on Ramachandran Plot <h1>

In [ ]:
import numpy as np
import pandas as pd
from glob import glob
import re
import os


qmsm_dir='./qMSM/'
trajdir='./trajs/'
rama=[]
#Loading torisonal angles data
for i in range(0,100):
    temp=np.loadtxt("{}rama/{}.txt".format(trajdir,i))[::10]
    rama.append(temp)
lumped_trajs=np.load("{}lumping_assignment.npy".format(qmsm_dir))[::10]

rama=np.array(rama)
rama=np.concatenate(rama)

#Saving torsional angles and the macrostate assignments into one file
df = pd.DataFrame([rama[:,0].tolist(),rama[:,1].tolist(),lumped_trajs.tolist()]).transpose()
df.columns = ['psi','phi','macrostate']
df.to_pickle("{}rama_macro.pickl".format(msm_dir))


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

df=pd.read_pickle("{}rama_macro.pickl".format(msm_dir))

plt.figure(figsize=(5,5))
macrostate_color={0:'#630C3A',1:'#39C8C6',2:'#D3500C',3:'#FFB139'}
sns.lmplot(data=df, x='psi', y='phi', hue='macrostate',palette=macrostate_color, 
           fit_reg=False, legend=True,legend_out=False,scatter_kws={'alpha':0.5})
plt.xlabel('$\psi$', fontsize=20)
plt.ylabel('$\phi$', fontsize=20)
plt.tight_layout()
plt.savefig("{}marcostate_scatter.png".format(msm_dir))

<h1>Outputing macrostate TPM for qMSM<h1>

In [ ]:
import numpy as np
from msmbuilder.msm import MarkovStateModel


def prep_macroTPM(initial_lagtime,ending_lagtime,delta_time,md_timestep,qmsm_dir='./qMSM/'):
    macro_tpm_lagtime=np.arange(initial_lagtime/md_timestep,
                                ending_lagtime/md_timestep+delta_time/md_timestep,
                                delta_time/md_timestep)
    """
    Outputting a file containing flatten macrostate TPMs calculated at different lag time(tau),
    separated by a user-defined time interval(delta_t).
    Each row in the output file represents one TPM generated at tau=n*delta_t
    ----------
    qmsm_dir: str
        Directory to store the output files
        
    initial_lagtime: int
        The starting lag time to output TPM
    
    ending_lagtime: int
        The final lag time to output TPM
        
    delta_time: int
        The time interval(share the same unit as timesetep) between each TPM 
    
    md_timestep: int
        Time step of the MD trajectories(in unit of picosecond in this tutorial)
    """
    macro_tpm=[]
    lumped_trajs=np.load("{}lumping_assignment.npy".format(qmsm_dir))
    for i in macro_tpm_lagtime:
        macro_msm=MarkovStateModel(verbose=False, lag_time=int(i), reversible_type='transpose')
        msm_macro=macro_msm.fit(lumped_trajs)
        macro_tpm.append(msm_macro.transmat_.flatten())

    macro_tpm=np.array(macro_tpm)
    np.save("{}macrostate_TPM_{}to{}.npy".format(qmsm_dir,initial_lagtime,ending_lagtime),macro_tpm)
    np.savetxt("{}macrostate_TPM_{}to{}.txt".format(qmsm_dir,initial_lagtime,ending_lagtime),
               macro_tpm,fmt='%.18e')

qmsm_dir='./qMSM/'
prep_macroTPM(initial_lagtime=0.1,ending_lagtime=50,delta_time=0.1,md_timestep=0.1)